<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Major Neural Network Architectures Challenge
## *Data Science Unit 4 Sprint 3 Challenge*

In this sprint challenge, you'll explore some of the cutting edge of data science. This week we studied several famous neural network architectures: 
recurrent neural networks (RNNs), long short-term memory (LSTMs), convolutional neural networks (CNNs), and Generative Adverserial Networks (GANs). In this sprint challenge, you will revisit these models. Remember, we are testing your knowledge of these architectures, not your ability to fit a model with high accuracy. 

__*Caution:*__  These approaches can be pretty heavy computationally. All problems were designed so that you should be able to achieve results within at most 5-10 minutes of runtime on Colab or a comparable environment. If something is running longer, double check your approach!

## Challenge Objectives
*You should be able to:*
* <a href="#p1">Part 1</a>: Train a RNN classification model.
* <a href="#p2">Part 2</a>: Utilize a pre-trained CNN for objective detection.
* <a href="#p3">Part 3</a>: Describe the components of an autoencoder.
* <a href="#p4">Part 4</a>: Describe yourself as a data scientist and elucidate your vision of AI.

<a id="p1"></a>
## Part 1 - RNNs

Use an RNN/LSTM to fit a multi-class classification model on reuters news articles to distinguish topics of articles. The data is already encoded properly for use in an RNN model. 

Your Tasks: 
- Use Keras to fit a predictive model, classifying news articles into topics. 
- Report your overall score and accuracy

For reference, the [Keras IMDB sentiment classification example](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py) will be useful, as well the RNN code we used in class.

__*Note:*__  Focus on getting a running model, not on maximizing accuracy with extreme data size or epoch numbers. Only revisit and push accuracy if you get everything else done!

In [ ]:
# Import libraries.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats

from google_images_download import google_images_download
from sklearn.metrics import accuracy_score

from tensorflow.keras.datasets import reuters
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import decode_predictions
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image, sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

In [ ]:
# Load news article dataset.
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=723812,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

In [ ]:
# Print encoding examples.
word_index = reuters.get_word_index(path="reuters_word_index.json")

print(f"Iran is encoded as {word_index['iran']} in the data.")
print(f"London is encoded as {word_index['london']} in the data.")
print("Words are encoded as numbers in our dataset.")

In [ ]:
# Define topics.
topic_indexes = pd.DataFrame.from_dict(data={'copper': 6, 
                                             'livestock': 28, 
                                             'gold': 25, 
                                             'money-fx': 19, 
                                             'ipi': 30, 
                                             'trade': 11, 
                                             'cocoa': 0, 
                                             'iron-steel': 31, 
                                             'reserves': 12, 
                                             'tin': 26, 
                                             'zinc': 37, 
                                             'jobs': 34, 
                                             'ship': 13, 
                                             'cotton': 14, 
                                             'alum': 23, 
                                             'strategic-metal': 27, 
                                             'lead': 45, 
                                             'housing': 7, 
                                             'meal-feed': 22, 
                                             'gnp': 21, 
                                             'sugar': 10, 
                                             'rubber': 32, 
                                             'dlr': 40,
                                             'veg-oil': 2,  
                                             'interest': 20, 
                                             'crude': 16, 
                                             'coffee': 9, 
                                             'wheat': 5, 
                                             'carcass': 15, 
                                             'lei': 35, 
                                             'gas': 41, 
                                             'nat-gas': 17, 
                                             'oilseed': 24, 
                                             'orange': 38, 
                                             'heat': 33, 
                                             'wpi': 43, 
                                             'silver': 42, 
                                             'cpi': 18, 
                                             'earn': 3, 
                                             'bop': 36, 
                                             'money-supply': 8, 
                                             'hog': 44, 
                                             'acq': 4, 
                                             'pet-chem': 39, 
                                             'grain': 1, 
                                             'retail': 29}, 
                                       orient='index',
                                       columns=['topic_id'])
                            
topic_indexes = topic_indexes.rename_axis('topic_name').reset_index()
topic_indexes = topic_indexes.set_index('topic_id').sort_index()
topic_indexes

In [ ]:
# Identify articles from training dataset with topic "orange".
np.where(y_train == 38)

In [ ]:
# Create reverse lookup dictionary for article reconstruction.
wordDict = {y:x for x, y in reuters.get_word_index().items()} 

# Reconstruct sample "orange" article.
print(' '.join([(wordDict.get(index-3)) for index in 
                filter(lambda i: (i >= 3), 
                       X_train[321])]))

In [ ]:
# Look at topic distribution in training dataset.
pd.merge(pd.DataFrame(y_train, columns=['topic_id']), 
         topic_indexes, 
         on='topic_id')['topic_name'].value_counts(normalize=True)

In [ ]:
# Check that all classes are represented in both datasets.
set(y_train) == set(y_test)

In [ ]:
# Check total number of words.
len(word_index.values())

In [ ]:
batch_size = 46
max_features = len(word_index.values()) + 1
maxlen = 200

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(set(y_train)), activation='softmax'))

In [ ]:
# You should only run this cell once your model has been
# properly configured.
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_train, y_train,
                            batch_size=batch_size,
                            verbose=False)
print('Train score:', score)
print('Train accuracy:', acc)

score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size,
                            verbose=False)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
# Define majority class from training dataset.
majority_class = scipy.stats.mode(y_train)[0][0]
majority_class

In [ ]:
# Calculate accuracy of majority classifier for comparison.
acc = accuracy_score(y_train, np.full_like(y_train, majority_class))
print('Majority classifier train accuracy:', acc)

acc = accuracy_score(y_test, np.full_like(y_test, majority_class))
print('Majority classifier test accuracy:', acc)

## Sequence Data Question
#### *Describe the `pad_sequences` method used on the training dataset. What does it do? Why do you need it?*

> The `pad_sequences` method ([API documentation](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences)) used on the training and test datasets truncates inputs longer than the maximum length specified and prepends zeros to any inputs shorter than that length. This is necessary because the architecture of a neural network is designed to work with inputs of a set length, but our raw data is presumably of variable length.

## RNNs versus LSTMs
#### *What are the primary motivations behind using Long-ShortTerm Memory Cell unit over traditional Recurrent Neural Networks?*

> Traditional RNNs are prone to vanishing and exploding gradients, which can cause slow convergence or failure to converge. They also heavily weight the most recent inputs, which means that they don't model long-term dependencies well. LSTMs are designed to address both of these issues.

## RNN / LSTM Use Cases
#### *Name and Describe 3 Use Cases of LSTMs or RNNs and why they are suited to that use case*

> LSTMs - and RNNs generally - are useful for prediction and generation tasks where inputs occur in a meaningful and/or relevant sequential context, because of their ability to retain and use information about that context, i.e. past inputs. Examples include machine translation, text generation, music composition, handwriting and speech recognition, robot control, and many kinds of time series prediction.

<a id="p2"></a>
## Part 2- CNNs

### Find the Frog

Time to play "find the frog!" Use Keras and ResNet50 (pre-trained) to detect which of the following images contain frogs:

<img align="left" src="https://d3i6fh83elv35t.cloudfront.net/newshour/app/uploads/2017/03/GettyImages-654745934-1024x687.jpg" width=400>


In [ ]:
# Download images.
response = google_images_download.googleimagesdownload()
arguments = {"keywords": "lily frog pond", "limit": 5, "print_urls": True}
absolute_image_paths = response.download(arguments)

In [ ]:
absolute_image_paths

At time of writing at least a few do, but since the Internet changes - it is possible your 5 won't. You can easily verify yourself, and (once you have working code) increase the number of images you pull to be more sure of getting a frog. Your goal is to validly run ResNet50 on the input images - don't worry about tuning or improving the model.

*Hint* - ResNet 50 doesn't just return "frog". The three labels it has for frogs are: `bullfrog, tree frog, tailed frog`

*Stretch goals* 
- Check for fish or other labels.
- Create a matplotlib visualization of the images and your prediction as the visualization label.

In [ ]:
def process_img_path(img_path):
    """
    Pre-process image for prediction with ResNet50.
    """
    img = image.load_img(img_path, target_size=(224, 224))
    image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

def predict_top_n(image, n):
    """
    Returns ResNet50's top n predictions for the given image.
    """
    model = ResNet50(weights='imagenet')
    features = model.predict(image)
    return decode_predictions(features, top=n)[0]

def img_contains_prob(img, name, top=10):
    """ Scans image for named object.
    
    Returns the estimated probability that the named object appears
    in the given image.
    
    Inputs:
    ---------
    img:  Precrossed image ready for prediction. The `process_img_path`
    function should already be applied to the image. 
    
    Returns: 
    ---------
    (float): [0, 1] - The probability that the named object appears.
    
    """
    model = ResNet50(weights='imagenet')
    features = model.predict(img)
    preds = decode_predictions(features, top=top)[0]
    
    probability = 0
    
    for pred in preds:
        if name in pred[1]:
            probability += pred[2]
            
    return probability

def img_contains(img, name, top=10):
    """ Scans image for named object.
    
    Should return a boolean (True/False) if the named object 
    is in the image with a predicted probability >= 0.5.
    
    Inputs:
    ---------
    img:  Precrossed image ready for prediction. The `process_img_path`
    function should already be applied to the image. 
    
    Returns: 
    ---------
    (boolean):  TRUE or FALSE - The named object appears in the image.    
    """
           
    probability = img_contains_prob(img, name)
    return probability >= 0.5

In [ ]:
# Check downloaded images for frogs.
for path in absolute_image_paths[0]['lily frog pond']:
    img = process_img_path(path)
    print(img_contains(img, 'frog'))

In [ ]:
# Examine underlying probability estimates.
for path in absolute_image_paths[0]['lily frog pond']:
    img = process_img_path(path)
    print(img_contains_prob(img, 'frog'))

In [ ]:
frog_path = ('C:\\Users\\aloha\\Downloads\\Elizabeth\\Lambda School\\'
            'downloads\\lily frog pond\\test_frog.jpg')
test_frog = process_img_path(frog_path)

In [ ]:
plt.imshow(image.load_img(frog_path, target_size=(224, 224)))
plt.axis('off');

In [ ]:
img_contains(test_frog, 'frog')

In [ ]:
img_contains_prob(test_frog, 'frog')

In [ ]:
img_contains_prob(test_frog, 'fish')

In [ ]:
fish_path = ('C:\\Users\\aloha\\Downloads\\Elizabeth\\Lambda School\\'
            'downloads\\lily frog pond\\test_fish.jpg')
test_fish = process_img_path(fish_path)

In [ ]:
plt.imshow(image.load_img(fish_path, target_size=(224, 224)))
plt.axis('off');

In [ ]:
img_contains(test_fish, 'fish')

In [ ]:
img_contains_prob(test_fish, 'fish')

In [ ]:
predict_top_n(test_fish, 10)

So we got lucky on this particular test image, but in general it looks like we'd need to know all of the fish species for which ResNet-50 has categories to effectively use this approach.

In [ ]:
img_contains(test_fish, 'frog')

In [ ]:
img_contains_prob(test_fish, 'frog')

In [ ]:
combo_path = ('C:\\Users\\aloha\\Downloads\\Elizabeth\\Lambda School\\'
              'downloads\\lily frog pond\\frog_turtle.jpg')
test_combo = process_img_path(combo_path)

In [ ]:
plt.imshow(image.load_img(combo_path, target_size=(224, 224)))
plt.axis('off');

In [ ]:
img_contains(test_combo, 'frog')

In [ ]:
img_contains_prob(test_combo, 'frog')

In [ ]:
img_contains(test_combo, 'turtle')

In [ ]:
img_contains_prob(test_combo, 'turtle')

In [ ]:
# What DOES ResNet50 think this is a picture of?
predict_top_n(test_combo, 10)

ResNet50 doesn't so much look for what an image _contains_ as what it is _of_ - it's easily confused by images with multiple distinct subjects.

#### Stretch Goal: Displaying Predictions

<a id="p3"></a>
## Part 3 - Autoencoders

Describe a use case for an autoencoder given that an autoencoder tries to predict its own input. 

__*Your Answer:*__ 

<a id="p4"></a>
## Part 4 - More...

Answer the following questions, with a target audience of a fellow Data Scientist:

- What do you consider your strongest area, as a Data Scientist?
- What area of Data Science would you most like to learn more about, and why?
- Where do you think Data Science will be in 5 years?
- What are the threats posed by AI to our society?
- How do you think we can counteract those threats? 
- Do you think achieving General Artifical Intelligence is ever possible?

A few sentences per answer is fine - only elaborate if time allows.

## Congratulations! 

Thank you for your hard work, and congratulations! You've learned a lot, and you should proudly call yourself a Data Scientist.


In [ ]:
from IPython.display import HTML

HTML("""<iframe src="https://giphy.com/embed/26xivLqkv86uJzqWk" width="480" height="270" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/mumm-champagne-saber-26xivLqkv86uJzqWk">via GIPHY</a></p>""")